# Introduction LangChain with OpenAI

### Loading OpenAI API Key

In [1]:
from dotenv import load_dotenv
import os

from langchain_core.output_parsers import StrOutputParser

# Load the .env file
load_dotenv(dotenv_path='../../.env')  # Specify the path to your .env file

# Access the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Check if the variable is loaded
if api_key or api_key == "":
    print("API key loaded successfully.")
else:
    print("Failed to load API key.")

API key loaded successfully.


### Initializing LLM

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage

llm = ChatOpenAI(
    model= "gpt-4o-mini",
    api_key= api_key,
)

response: AIMessage = llm.invoke("What is LangChain in one sentence like I am 5?")
print(response.content)

LangChain is like a special tool that helps computers understand and make stories or chat with people using their own words!


## Prompt Templates and Chaining

In [4]:
from langchain_core.prompts import PromptTemplate

template = "You are a book recommender. Recommend a book for {topic}."

prompt = PromptTemplate.from_template(
    template= template,
)

llm_chain = prompt | llm

topic = "Discrepancies between what is taught in US schools and what is taught in Brazilian schools about Amazon Rainforest"
print(llm_chain.invoke({"topic":topic}).content)

For understanding the discrepancies between what is taught in U.S. schools and Brazilian schools about the Amazon Rainforest, I recommend **"The Amazon: What Everyone Needs to Know" by David Hill**. 

This book provides a comprehensive overview of the Amazon Rainforest, covering its ecological importance, cultural significance, and the various environmental issues it faces. It offers insights into how the Amazon is perceived in different parts of the world and contrasts educational narratives. 

Additionally, if you're looking for a more academic focus, **"The Amazon: A Natural History" by Norman Myers** offers a thorough exploration of the biodiversity and environmental issues of the Amazon, suitable for understanding the content differences in educational contexts.

Both books can provide perspective on the geographic, ecological, and socio-economic factors that shape the teaching of the Amazon Rainforest in different educational systems.


## Chat Prompt Templates

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert that returns the colors present in a country's flag."),
        ("human", "France"),
        ("ai", "blue, white, red"),
        ("human", "{country}")
    ]
)

llm_chain = prompt_template | llm

country = "Japan"
response = llm_chain.invoke({"country": country})
print(response.content)

red, white


## Few Shot Prompt Template

In [8]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

examples = [
    {
        "question": "What is the capital of France?",
        "answer": "Paris"
    },
    {
        "question": "What is the capital of Brazil?",
        "answer": "Brasilia"
    },
    {
        "question": "What is the capital of Japan?",
        "answer": "Tokyo"
    }
]

example_prompt = PromptTemplate.from_template("Question: {question}\nAnswer: {answer}")

prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a geography expert that returns the capital of a country.",
    suffix="\nQuestion: {question}\nAnswer:",
    input_variables=["question"],
)

llm_chain = prompt_template | llm

response = llm_chain.invoke({
        "question": "What is the capital of Italy?"
    })
print(response.content)

Rome


## Sequential Chain Prompt Template

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create a prompt template that takes an input activity
# This defines the first prompt that asks for a learning plan based on an activity
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
# This second prompt will take the output from the first prompt and add a time constraint
time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Complete the sequential chain with LCEL (LangChain Expression Language)
# This creates a pipeline of operations that will execute in sequence
seq_chain = (
    {"learning_plan": learning_prompt | llm | StrOutputParser()}  # First: Pass activity to learning_prompt, send to LLM, parse to string
    | time_prompt  # Second: Feed that result into the time_prompt as "learning_plan"
    | llm  # Third: Send the time-constrained prompt to the LLM
    | StrOutputParser()  # Finally: Convert the LLM's response to a string
)

# Invoke the seq chain with an activity
response = seq_chain.invoke({"activity": "play golf"})
print(response)

Here's a focused 7-day plan to help you kickstart your journey into learning how to play golf. Each day encompasses several tasks aimed at ensuring you grasp the basics by the end of the week.

### Day 1: Understand the Basics
- **Rules & Etiquette**: Spend 30 minutes reading about the basic rules of golf and golf etiquette. Numerous resources are available online, including the USGA website.
- **Equipment Knowledge**: Watch a YouTube video or read articles about different types of clubs. Familiarize yourself with drivers, irons, putters, tees, and balls.

### Day 2: Get the Right Equipment 
- **Visit a Golf Store**: Spend time at a local golf shop to look at beginner sets. If possible, get fitted for clubs to ensure they suit your height and swing. 
- **Purchase Essentials**: Buy at least a half set of clubs (driver, a few irons, and a putter), as well as golf balls, tees, and gloves, while keeping your budget in mind.

### Day 3: Take Lessons
- **Book a Lesson**: Schedule an introduc